In [ ]:
#Pyspark-SQL-Reading from files

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('spark-SQL').master('local[1]').getOrCreate()

In [2]:
df = spark.read.csv('C:\\Users\\Raju\\Documents\\survey-2020.csv')

In [3]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)



In [4]:
df.show()

+----+--------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+---------+--------------------+
| _c0|                 _c1|                 _c2|                 _c3|               _c4|          _c5|                 _c6|                 _c7|      _c8|                 _c9|
+----+--------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+---------+--------------------+
|Year|Industry_aggregat...|Industry_code_NZSIOC|Industry_name_NZSIOC|             Units|Variable_code|       Variable_name|   Variable_category|    Value|Industry_code_ANZ...|
|2020|             Level 1|               99999|      All industries|Dollars (millions)|          H01|        Total income|Financial perform...|  733,258|ANZSIC06 division...|
|2020|             Level 1|               99999|      All industries|Dollars (millions)|          H04|Sales, government.

In [9]:
df1 = spark.read.format('csv').option('header','true').load('C:\\Users\\Raju\\Documents\\survey-2020.csv')

In [10]:
df1.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Industry_aggregation_NZSIOC: string (nullable = true)
 |-- Industry_code_NZSIOC: string (nullable = true)
 |-- Industry_name_NZSIOC: string (nullable = true)
 |-- Units: string (nullable = true)
 |-- Variable_code: string (nullable = true)
 |-- Variable_name: string (nullable = true)
 |-- Variable_category: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Industry_code_ANZSIC06: string (nullable = true)



In [11]:
df1.show()

+----+---------------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+---------+----------------------+
|Year|Industry_aggregation_NZSIOC|Industry_code_NZSIOC|Industry_name_NZSIOC|             Units|Variable_code|       Variable_name|   Variable_category|    Value|Industry_code_ANZSIC06|
+----+---------------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+---------+----------------------+
|2020|                    Level 1|               99999|      All industries|Dollars (millions)|          H01|        Total income|Financial perform...|  733,258|  ANZSIC06 division...|
|2020|                    Level 1|               99999|      All industries|Dollars (millions)|          H04|Sales, government...|Financial perform...|  660,630|  ANZSIC06 division...|
|2020|                    Level 1|               99999|      All industries

In [12]:
print(type(df1))

<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
#File-Formats
Text(csv,pipe delimited,tab delimited)
Avro-
Parquet - Columnar storage (compresson)
orc -optimised Row columnar Format (updaet/Delete) [Hive
Json - 

In [15]:
from pyspark.sql.functions import col

dfSelect = df1.select(col('Year'),col('Variable_code'),col('Industry_name_NZSIOC'),col('Variable_category'),col('Value'))
dfSelect.show()

+----+-------------+--------------------+--------------------+---------+
|Year|Variable_code|Industry_name_NZSIOC|   Variable_category|    Value|
+----+-------------+--------------------+--------------------+---------+
|2020|          H01|      All industries|Financial perform...|  733,258|
|2020|          H04|      All industries|Financial perform...|  660,630|
|2020|          H05|      All industries|Financial perform...|   54,342|
|2020|          H07|      All industries|Financial perform...|   18,285|
|2020|          H08|      All industries|Financial perform...|  654,872|
|2020|          H09|      All industries|Financial perform...|   32,730|
|2020|          H10|      All industries|Financial perform...|    7,509|
|2020|          H11|      All industries|Financial perform...|   26,821|
|2020|          H12|      All industries|Financial perform...|  119,387|
|2020|          H13|      All industries|Financial perform...|      305|
|2020|          H14|      All industries|Financial 

In [21]:
dfVarCategories = dfSelect.select(col('Variable_category')).distinct()
dfVarCategories.show()
dfSelect.printSchema()

+--------------------+
|   Variable_category|
+--------------------+
|Financial perform...|
|  Financial position|
|    Financial ratios|
+--------------------+

root
 |-- Year: string (nullable = true)
 |-- Variable_code: string (nullable = true)
 |-- Industry_name_NZSIOC: string (nullable = true)
 |-- Variable_category: string (nullable = true)
 |-- Value: string (nullable = true)



In [24]:
dfCast = dfSelect.withColumn("Value",col('Value').cast('Integer'))

In [32]:
dfCast.groupBy(col('Variable_category')).sum('Value') \
     .select(col('Variable_category'),col('sum(Value)').alias('Total_Value')).show()

+--------------------+-----------+
|   Variable_category|Total_Value|
+--------------------+-----------+
|Financial perform...|    2193723|
|  Financial position|    1412434|
|    Financial ratios|     311989|
+--------------------+-----------+



In [33]:
df1.createOrReplaceTempView('Survey')

In [37]:
query = '''select Variable_category,sum(cast(Value as int)) as Total_Value, min(cast(Value as int)) as mn, max(cast(Value as int)) as mx 
from Survey group by Variable_category'''

In [38]:
result = spark.sql(query)
result.show()

+--------------------+-----------+----+---+
|   Variable_category|Total_Value|  mn| mx|
+--------------------+-----------+----+---+
|Financial perform...|    2193723|-810|999|
|  Financial position|    1412434|   1|999|
|    Financial ratios|     311989|-600|500|
+--------------------+-----------+----+---+

